# `yield from` 语法具体实现

https://peps.python.org/pep-0380/


```Python
# EXPR 指的是任意表达式，它的值必须是一个可迭代对象
RESULT = yield from EXPR
```

In [ ]:
# 简化1

_i = iter(EXPR)  # __iter__ -> __await__
try:
    _y = _i.send(None) # prime
except StopIteration as _e: # 直接结束，一次 yield 都没遇到
    _r = _e.value
else:
    while 1: # 不遇到 StopIteration 就一直循环
        try:
            _s = yield _y # 招原样 yield 回去，并接受 send 传入的值，保存到 _s
        except GeneratorExit as _e: # 处理 close
            _i.close
            raise _e
        except BaseException as _e: # 处理其他异常
            _x = sys.exc_info()
            try:
                _y = _i.throw(*_x)
            except StopIteration as _e:
                _r = _e.value
                break
        else:
            try:
                _y = _i.send(_s)  # 接受到的值原样再 send 下去
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r  # StopIteration 后的值就是结果

In [ ]:
# 简化2

_i = iter(EXPR)  # __iter__ -> __await__
while 1: # 不遇到 StopIteration 就一直循环
    try:
        _y = _i.send(None) # 总是 None
    except StopIteration as _e: # 直接结束，一次 yield 都没遇到
        _r = _e.value
        break
    else:
        try:
            yield _y # 照原样 yield 回去，因为传入的是 None, 因此不保存
        except GeneratorExit as _e: # 处理 close
            _i.close
            raise _e
        except BaseException as _e: # 处理其他异常
            _x = sys.exc_info()
            try:
                _y = _i.throw(*_x)
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r  # StopIteration 后的值就是结果

In [ ]:
# 简化3

_i = iter(EXPR)  # __iter__ -> __await__
while 1: # 不遇到 StopIteration 就一直循环
    try:
        _y = _i.send(None) # 总是 None
    except StopIteration as _e: # 直接结束，一次 yield 都没遇到
        _r = _e.value
        break
    else:
        yield _y # 照原样 yield 回去，因为传入的是 None, 因此不保存

RESULT = _r  # StopIteration 后的值就是结果

# 例子

![](./代码演进.png)

## 一个同步模式的简单任务

In [1]:
def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_result = big_step()

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [2]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    small_result = small_step()
    print("     end small_step")
    return small_result * 1000

In [3]:
def small_step():
    print("         working on small step")
    return 123

In [4]:
# 执行任务
one_task()

begin task
     begin big_step
     begin small_step
         working on small step
     end small_step
     end big_step with 123000
end task


## 遇到阻塞了

In [5]:
import time

def small_step():
    print("         sleeping for 2 seconds")
    time.sleep(2)
    print("         working on small step")
    return 123

In [6]:
# 执行任务
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
         working on small step
     end small_step
     end big_step with 123000
end task


## 听说 `yeild` 变协程可以不阻塞

In [7]:
import time

def small_step():
    print("         sleeping for 2 seconds")
    yield time.sleep(2)
    print("         working on small step")
    return 123

In [8]:
one_task()
# 会报错，因为 yield 的函数是生成器，没法直接相乘

begin task
     begin big_step
     begin small_step
     end small_step


TypeError: unsupported operand type(s) for *: 'generator' and 'int'

## `yield` 有传染性

In [ ]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    # small_result = small_step()
    # 因为 small_step 是一个协程，因此要这样处理
    small_coro = small_step()
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            # 处理协程返回值
            small_result = e.value
            break
        else:
            ...

    print("     end small_step")
    return small_result * 1000

In [10]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
         working on small step
     end small_step
     end big_step with 123000
end task


## 将阻塞从下游传到上游

In [ ]:
def small_step():
    print("         sleeping for 2 seconds")
    # 不在这里执行 sleep，而是将阻塞从下游传递到上游
    t1 = time.time()
    yield time.sleep, 2
    t2 = time.time()
    assert t2 - t1 > 2, "睡眠时间不足"
    print("         working on small step")
    return 123

In [12]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds


AssertionError: 睡眠时间不足

In [ ]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    # small_result = small_step()
    # 因为 small_step 是一个协程，因此要这样处理
    small_coro = small_step()
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            # 处理协程返回值
            small_result = e.value
            break
        else:
            # 将 small_coro 的值向上传递
            yield x

    print("     end small_step")
    return small_result * 1000

In [14]:
one_task()
# 由于 big_step 中使用了 yield, 因此 big_step 是一个生成器函数, 因此 one_task 函数也要做如下修改

begin task
     begin big_step
     end big_step with <generator object big_step at 0x000001E68F2DD2A0>
end task


In [15]:
def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_coro = big_step()
    while True:
        try:
            x = big_coro.send(None)
        except StopIteration as e:
            # 处理协程返回值
            big_result = e.value
            break
        else:
            print(f"     got {x}")
            # 这里调用返回的函数
            func, args = x
            func(args)

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [16]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got (<built-in function sleep>, 2)
         working on small step
     end small_step
     end big_step with 123000
end task


## 阶段总结

- 协程自己并不能消除阻塞
- 协程具有传染性
- 协程通过yield把阻塞换个方式传递给了上游
- 最终阻塞仍然需要被解决

## `yield from` 来帮忙

In [ ]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    # small_result = small_step()

    # yield from 代替上面和下面的代码
    small_result = yield from small_step()

    # 因为 small_step 是一个协程，因此要这样处理
    # small_coro = small_step()
    # while True:
    #     try:
    #         x = small_coro.send(None)
    #     except StopIteration as e:
    #         # 处理协程返回值
    #         small_result = e.value
    #         break
    #     else:
    #         # 将 small_coro 的值向上传递
    #         yield x

    print("     end small_step")
    return small_result * 1000

In [18]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got (<built-in function sleep>, 2)
         working on small step
     end small_step
     end big_step with 123000
end task


## 阶段总结2

为了方便讨论：
我们将最未端的遇到阻塞而不得不主动 `yie1d` 的协程称之为「主动协程」

中间接受到下游的传导而不得不跟随着 `yield` 的协程称之为「被动协程」

「主动协程」是最先出栈的位置「被动协程」可能有很多层

`yield from` 大大简化了「被动协程」的编码


## `yield from` 一统江湖

In [ ]:
def small_step():
    print("         sleeping for 2 seconds")
    # 不在这里执行 sleep，而是将阻塞从下游传递到上游
    t1 = time.time()

    # yield time.sleep, 2
    yield from YieldFromAble((time.sleep, 2))

    t2 = time.time()
    assert t2 - t1 > 2, "睡眠时间不足"
    print("         working on small step")
    return 123

In [ ]:
# 定义辅助 class, 用来包裹 yield 内容, 实现 yield from 调用原本 yield 的内容
class YieldFromAble:
    def __init__(self, obj):
        self.value = obj

    def __iter__(self):
        yield self.value

In [21]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got (<built-in function sleep>, 2)
         working on small step
     end small_step
     end big_step with 123000
end task


## 阶段总结3

通过一个 `YieldFromAble` 对象，将最末端的 `yield` 进行封装，把协程的调用方式统一成了 `yield from`。


## 将任务彻底协程化

In [22]:
def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    # big_coro = big_step()
    # while True:
    #     try:
    #         x = big_coro.send(None)
    #     except StopIteration as e:
    #         # 处理协程返回值
    #         big_result = e.value
    #         break
    #     else:
    #         print(f"     got {x}")
    #         # 这里调用返回的函数
    #         func, args = x
    #         func(args)

    big_result = yield from big_step()


    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [24]:
# 这样没法运行，因为 one_task() 是一个生成器函数，不能直接调用。
one_task()

<generator object one_task at 0x000001E68F32C2B0>

## 一个通用的任务驱动器

In [33]:
# 定义辅助 class, 用来包裹 yield 内容, 实现 yield from 调用原本 yield 的内容
class YieldFromAble:
    def __init__(self, obj):
        self.value = obj

    def __iter__(self):
        # yield self.value 改为 yield self.value
        yield  self

In [36]:
class Task:
    def __init__(self, coro) -> None:
        self.coro = coro

    def run(self):
        print("----------")
        while True:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                # 处理协程返回值
                result = e.value
                break
            else:
                print(f"     got {x}")
                # 上面改造 YieldFromAble yield self 之后，可以保证这里的 x 一定是 YieldFromAble 类型
                assert isinstance(x, YieldFromAble)
                # 这里调用返回的函数
                func, args = x.value
                func(args)
        print("----------")


In [37]:
t = Task(one_task())
t.run()

----------
begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got <__main__.YieldFromAble object at 0x000001E68ED168D0>
         working on small step
     end small_step
     end big_step with 123000
end task
----------


## 完成了整个任务的协程化改造

In [38]:
def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_result = yield from big_step()

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [39]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")

    small_result = yield from small_step()

    print("     end small_step")
    return small_result * 1000

In [ ]:
def small_step():
    print("         sleeping for 2 seconds")
    # 不在这里执行 sleep，而是将阻塞从下游传递到上游
    t1 = time.time()

    yield from YieldFromAble((time.sleep, 2))

    t2 = time.time()
    assert t2 - t1 > 2, "睡眠时间不足"
    print("         working on small step")
    return 123

In [41]:
t = Task(one_task())
t.run()

----------
begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got <__main__.YieldFromAble object at 0x000001E68ED4F950>
         working on small step
     end small_step
     end big_step with 123000
end task
----------


## 阶段总结4

通过在首尾两端分别添加一个通用的组件，完成了整个任务的协程化更新。

可以看到在 `yield from` 的帮助下，被动协程的部分代码几乎不用改变。

只有最未端要主动 `yield` 的地方变化比较大。